In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
init_notebook_mode(connected=True)

In [2]:
cols = ['load_date', 'timestamp', 'date','exchange','symbol','price','volume','type','side']
df = pd.read_csv("data/history/Binance.ETH-USDT.csv", 
                 names=cols, 
                 parse_dates=['load_date', 'date'],
                 dtype={'exchange':'category',
                        'side':'category',
                        'type':'category',
                        'symbol':'category'
                       }
                ).set_index('date')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 425821 entries, 2018-05-17 12:36:35.443000 to 2018-05-22 06:15:55.594000
Data columns (total 8 columns):
load_date    425821 non-null datetime64[ns]
timestamp    425821 non-null int64
exchange     425821 non-null category
symbol       425821 non-null category
price        425821 non-null float64
volume       425821 non-null float64
type         425821 non-null category
side         425821 non-null category
dtypes: category(4), datetime64[ns](1), float64(2), int64(1)
memory usage: 17.9 MB


In [4]:
# localize
df.index = df.index.tz_localize('UTC').tz_convert('Europe/Moscow')

In [5]:
df = df[['exchange','symbol','price','volume','side']]
df.tail(10)

,exchange,symbol,price,volume,side
date,,,,,
2018-05-22 09:15:53.347000+03:00,Binance,ETH/USDT,694.37,0.37240,buy
2018-05-22 09:15:53.441000+03:00,Binance,ETH/USDT,694.01,0.01194,sell
2018-05-22 09:15:53.441000+03:00,Binance,ETH/USDT,694.00,21.22563,sell
2018-05-22 09:15:54.639000+03:00,Binance,ETH/USDT,694.01,0.06850,buy
2018-05-22 09:15:54.639000+03:00,Binance,ETH/USDT,694.01,0.06850,buy
2018-05-22 09:15:54.648000+03:00,Binance,ETH/USDT,694.01,1.00000,buy
2018-05-22 09:15:54.695000+03:00,Binance,ETH/USDT,694.01,0.12005,buy
2018-05-22 09:15:54.791000+03:00,Binance,ETH/USDT,694.01,0.07540,buy
2018-05-22 09:15:54.246000+03:00,Binance,ETH/USDT,694.01,0.12580,buy


In [6]:
df[['price','volume']].describe()

,price,volume
count,425821.000000,425821.000000
mean,694.902048,1.063158
std,15.498098,6.159553
min,656.000000,0.000010
25%,683.870000,0.034370
50%,694.960000,0.201000
75%,707.730000,0.708440
max,729.880000,1305.808330


In [13]:
#df.resample('60min', how={'price':'ohlc', 'volume':'sum'})
df_hours = df.resample('60min').agg({'price':'ohlc', 'volume':'sum'})

In [14]:
df_hours.tail()

price                              volume
                             open    high     low   close      volume
date                                                                 
2018-05-22 05:00:00+03:00  686.83  691.23  685.00  687.51  3140.64372
2018-05-22 06:00:00+03:00  688.46  689.62  683.11  688.67  4497.97111
2018-05-22 07:00:00+03:00  688.76  689.00  684.15  687.39  2570.49603
2018-05-22 08:00:00+03:00  687.90  690.76  686.82  688.72  2683.41426
2018-05-22 09:00:00+03:00  688.71  695.95  688.22  694.37  1396.16455

### Graph

In [15]:
data = [dict( type='candlestick',
              #x=df_hours.index.tz_localize('Europe/Moscow'), 
              x=df_hours.index + pd.DateOffset(hours=3),
              open=df_hours.price.open,
              high=df_hours.price.high,
              low=df_hours.price.low,
              close=df_hours.price.close,
              #volume=df_hours.volume.volume
              yaxis = 'y2',
              name = 'OHLC',
             )]

In [16]:
def movingaverage(interval, window_size=10):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

window_size = 5
mv_y = movingaverage(df_hours.price.close, window_size=window_size)
mv_x = list(df_hours.index)

# Clip the ends
mv_x = mv_x[window_size:-window_size]
mv_y = mv_y[window_size:-window_size]

In [17]:
def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std  = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)
    return rolling_mean, upper_band, lower_band

bb_avg, bb_upper, bb_lower = bbands(df_hours.price.close)

In [19]:
layout = go.Layout(xaxis = dict(rangeslider = dict(visible = False)))
fig = go.Figure(data=data, layout=layout)

# OHLC
fig['data'] = data

# Volume
fig['data'].append( dict(x=df_hours.index + pd.DateOffset(hours=3), 
                         y=df_hours.volume.volume,           
                         type='bar',
                         yaxis='y', 
                         name='Volume',
                        ) )
# MA
fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

# Bollinger bands
fig['data'].append( dict( x=df_hours.index, y=bb_upper, type='scatter', yaxis='y2', 
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none', 
                         legendgroup='Bollinger Bands', name='Bollinger Bands') )

fig['data'].append( dict( x=df_hours.index, y=bb_lower, type='scatter', yaxis='y2',
                         line = dict( width = 1 ),
                         marker=dict(color='#ccc'), hoverinfo='none',
                         legendgroup='Bollinger Bands', showlegend=False ) )

fig['layout']['title'] = "{} {}".format(df.exchange[0].capitalize(), df.symbol[0])
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
#fig['layout']['xaxis'] = dict( rangeselector = dict( visible = False ) )
#fig['layout']['xaxis'] = dict(showticklabels = True)
fig['layout']['yaxis'] = dict( domain = [0.0, 0.2], showticklabels = True )
fig['layout']['yaxis2'] = dict( domain = [0.2, 1.0] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.97, x=0.3, yanchor='top' )
fig['layout']['margin'] = dict( t=40, b=50, r=40, l=40)

iplot(fig, filename='OHLC without Rangeslider', validate=False)